## **Setting up Development Environment**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/HL

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/HL


In [4]:
! pip install -U -q transformers
! pip install -U -q sentencepiece

     |████████████████████████████████| 2.9 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 51.7 MB/s 
     |████████████████████████████████| 56 kB 3.0 MB/s 
     |████████████████████████████████| 636 kB 41.9 MB/s 
     |████████████████████████████████| 895 kB 52.6 MB/s 
     |████████████████████████████████| 1.2 MB 5.2 MB/s 


## **Importing Libraries**

In [6]:
import pandas as pd

import numpy as np

import re

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from transformers import RobertaTokenizer, RobertaModel

import pickle

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## **Loading Data**

In [7]:
dataframe = pd.read_csv("Dataset/Tamil-Codemixed_offensive_data_Training-Tweet-HL.csv")

In [8]:
dataframe.head()

,id,text,category
0,TA_HL100,Iyaooo Kovam pattutene sothula visatha vachuru...,NOT
1,TA_HL101,@Asha Apo neenga atha government ku theriya pa...,NOT
2,TA_HL102,@Bala sundar ayyo sorry...antha line ah explai...,NOT
3,TA_HL105,@kalimuthu ne ena lusa...yaaru edhu panaalum e...,NOT
4,TA_HL109,1st baby ku neat ah feed panunga plzz ipdi iru...,NOT


In [9]:
dataframe.describe()

,id,text,category
count,4000,4000,4000
unique,3999,3657,2
top,TA_TW1600,avar nermaya sambadikira kaasu enna venalum pa...,NOT
freq,2,2,2020


In [10]:
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [11]:
dataframe.drop_duplicates(subset ="id",keep = False, inplace = True)
dataframe.describe()

,id,text,category
count,3998,3998,3998
unique,3998,3656,2
top,TA_TW2448,avar nermaya sambadikira kaasu enna venalum pa...,NOT
freq,1,2,2020


In [12]:
dataframe.shape

(3998, 3)

In [13]:
text = dataframe['text']
text

0       Iyaooo Kovam pattutene sothula visatha vachuru...
1       @Asha Apo neenga atha government ku theriya pa...
2       @Bala sundar ayyo sorry...antha line ah explai...
3       @kalimuthu ne ena lusa...yaaru edhu panaalum e...
4       1st baby ku neat ah feed panunga plzz ipdi iru...
                              ...                        
3995    Yaroda body structure semaya irukum? Sema mood...
3996    Yenda naangala politics varom nu pala varusham...
3997    Yepdithan seruppala adichalum arasiyalvathikku...
3998    @USER Paithiyam ena unga vanthu full ah forwar...
3999    RT @USER : Itha vidaa kevalam veraa irukaa vij...
Name: text, Length: 3998, dtype: object

In [14]:
label = dataframe['category']
label

0       NOT
1       NOT
2       NOT
3       NOT
4       NOT
       ... 
3995    OFF
3996    OFF
3997    OFF
3998    OFF
3999    OFF
Name: category, Length: 3998, dtype: object

## **Data Preprocessing**

In [15]:
le = LabelEncoder()
label = le.fit_transform(label)

In [16]:
english_stopwords = stopwords.words("english")
english_stemmer = SnowballStemmer("english")

def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+",' ', text)
    text = re.sub(r"#[A-Za-z0-9]+",' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+",' ', text)
    text = re.sub("RT ", " ", text)
    text = re.sub("\n", " ", text)
    text = re.sub(r" +", " ", text)
    tokens = []
    for token in text.split():
        if token not in english_stopwords:
            token = english_stemmer.stem(token)
            tokens.append(token)
    return " ".join(tokens)

text = [clean_text(i) for i in text]

text = np.array(text)

## **Feature Extraction**

In [17]:
# Load Transformer Model

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
# Sample Output

tokenized_input = tokenizer(
        text[0],
        padding=True,
        truncation=True,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [19]:
print(tokenized_input)

{'input_ids': tensor([[    0, 10252, 41242,   449,  1417,   424,   181,  2611, 19995,   579,
          6157,  5571, 17737, 20647,   748,  1488,  5515,  9965, 39029, 29942,
          1020,  3036,   734,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [20]:
print(sample_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0479,  0.0386, -0.0235,  ..., -0.0607, -0.0681,  0.0071],
         [-0.0424,  0.1548, -0.2476,  ...,  0.1079, -0.0624, -0.2119],
         [ 0.0639, -0.0626, -0.0158,  ..., -0.1299, -0.1554, -0.0059],
         ...,
         [ 0.0781, -0.1560,  0.1193,  ..., -0.0388, -0.0981,  0.0083],
         [ 0.1100, -0.2310,  0.0652,  ..., -0.1329, -0.0864,  0.2060],
         [-0.0399,  0.0270, -0.0442,  ..., -0.0976, -0.0660, -0.0078]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[ 1.6845e-02, -2.1110e-01, -2.3160e-01, -9.0754e-02,  1.3721e-01,
          1.8500e-01,  2.6948e-01, -8.5844e-02, -6.1398e-02, -1.6717e-01,
          2.5157e-01, -4.3901e-03, -1.0386e-01,  1.0079e-01, -1.5030e-01,
          4.9858e-01,  2.1782e-01, -4.7191e-01,  3.1932e-02, -2.4542e-02,
         -2.6461e-01,  6.1918e-02,  4.7698e-01,  3.0081e-01,  1.1124e-01,
          7.7561e-02, -1.5456e-01, -2.9800e-02,  1.8985e-01,  2.4303

In [21]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [22]:
# Tokenizing Input Data

input = []

for i in text:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [23]:
# Embedding the Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./XLM-Roberta-Embedded-TrainData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./XLM-Roberta-Embedded-TrainData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./XLM-Roberta-Embedded-TrainData/output1.pickle done
./XLM-Roberta-Embedded-TrainData/output2.pickle done
./XLM-Roberta-Embedded-TrainData/output3.pickle done
./XLM-Roberta-Embedded-TrainData/output4.pickle done
./XLM-Roberta-Embedded-TrainData/output5.pickle done
./XLM-Roberta-Embedded-TrainData/output6.pickle done
./XLM-Roberta-Embedded-TrainData/output7.pickle done
./XLM-Roberta-Embedded-TrainData/output8.pickle done
./XLM-Roberta-Embedded-TrainData/output9.pickle done
./XLM-Roberta-Embedded-TrainData/output10.pickle done
./XLM-Roberta-Embedded-TrainData/output11.pickle done
./XLM-Roberta-Embedded-TrainData/output12.pickle done
./XLM-Roberta-Embedded-TrainData/output13.pickle done
./XLM-Roberta-Embedded-TrainData/output14.pickle done
./XLM-Roberta-Embedded-TrainData/output15.pickle done
./XLM-Roberta-Embedded-TrainData/output16.pickle done
./XLM-Roberta-Embedded-TrainData/output17.pickle done
./XLM-Roberta-Embedded-TrainData/output18.pickle done
./XLM-Roberta-Embedded-TrainData/outp

In [24]:
# Loading Embedded Input Data from Disk

output = []

for i in range(20):
    file_name = "./XLM-Roberta-Embedded-TrainData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X = output
output = []

./XLM-Roberta-Embedded-TrainData/output1.pickle done
./XLM-Roberta-Embedded-TrainData/output2.pickle done
./XLM-Roberta-Embedded-TrainData/output3.pickle done
./XLM-Roberta-Embedded-TrainData/output4.pickle done
./XLM-Roberta-Embedded-TrainData/output5.pickle done
./XLM-Roberta-Embedded-TrainData/output6.pickle done
./XLM-Roberta-Embedded-TrainData/output7.pickle done
./XLM-Roberta-Embedded-TrainData/output8.pickle done
./XLM-Roberta-Embedded-TrainData/output9.pickle done
./XLM-Roberta-Embedded-TrainData/output10.pickle done
./XLM-Roberta-Embedded-TrainData/output11.pickle done
./XLM-Roberta-Embedded-TrainData/output12.pickle done
./XLM-Roberta-Embedded-TrainData/output13.pickle done
./XLM-Roberta-Embedded-TrainData/output14.pickle done
./XLM-Roberta-Embedded-TrainData/output15.pickle done
./XLM-Roberta-Embedded-TrainData/output16.pickle done
./XLM-Roberta-Embedded-TrainData/output17.pickle done
./XLM-Roberta-Embedded-TrainData/output18.pickle done
./XLM-Roberta-Embedded-TrainData/outp

In [25]:
X = np.array(X)
X.shape

(3998, 1, 768)

In [26]:
X = X.reshape(3998, 768)
X.shape

(3998, 768)

In [27]:
y = label
y.shape

(3998,)

## **Train Test Split**

In [28]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
X_train.shape

(3198, 768)

In [30]:
X_val.shape

(800, 768)

## **Logistic Regression**

In [31]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

              precision    recall  f1-score   support

           0       0.75      0.65      0.70       420
           1       0.66      0.76      0.71       380

    accuracy                           0.70       800
   macro avg       0.71      0.71      0.70       800
weighted avg       0.71      0.70      0.70       800



## **Support Vector Machine**

In [32]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.61      0.83      0.70       420
           1       0.68      0.40      0.51       380

    accuracy                           0.63       800
   macro avg       0.64      0.62      0.60       800
weighted avg       0.64      0.63      0.61       800



## **Naive Bayes**

In [33]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.66      0.56      0.61       420
           1       0.58      0.68      0.63       380

    accuracy                           0.62       800
   macro avg       0.62      0.62      0.62       800
weighted avg       0.63      0.62      0.62       800



## **Stochastic Gradient Descent**

In [34]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

              precision    recall  f1-score   support

           0       0.62      0.92      0.74       420
           1       0.81      0.37      0.51       380

    accuracy                           0.66       800
   macro avg       0.71      0.65      0.62       800
weighted avg       0.71      0.66      0.63       800



## **K Nearest Neighbours**

In [35]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.68      0.59      0.63       420
           1       0.60      0.70      0.65       380

    accuracy                           0.64       800
   macro avg       0.64      0.64      0.64       800
weighted avg       0.64      0.64      0.64       800



## **Decision Tree**

In [36]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.66      0.63      0.65       420
           1       0.61      0.64      0.63       380

    accuracy                           0.64       800
   macro avg       0.64      0.64      0.64       800
weighted avg       0.64      0.64      0.64       800



## **Random Forest**

In [37]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

              precision    recall  f1-score   support

           0       0.76      0.72      0.74       420
           1       0.71      0.75      0.73       380

    accuracy                           0.73       800
   macro avg       0.73      0.74      0.73       800
weighted avg       0.74      0.73      0.74       800



## **Majority Voting**

In [38]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    # Including only LR, SGD & RF in the predictions
    predictions = [lr_pred[i], rf_pred[i], sgd_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))

              precision    recall  f1-score   support

           0       0.73      0.75      0.74       420
           1       0.72      0.68      0.70       380

    accuracy                           0.72       800
   macro avg       0.72      0.72      0.72       800
weighted avg       0.72      0.72      0.72       800

